## Model

The table shows main model parameters for each experiment. 

In [ ]:
def summarize_models(model_list):
    summs = []
    for (model_id, config, csvdir) in model_list:
        coef_file = os.path.join(csvdir, '{}_betas.csv'.format(model_id))
        if os.path.exists(coef_file):
            coefs = pd.read_csv(coef_file)
            model_summary = pd.DataFrame({'model': [model_id],
                                          'N features': len(coefs),
                                          'N negative': len(coefs.ix[coefs['standardized']<0]),
                                          'learner': config['model'],
                                          'train_label': config['train_label_column']})
            summs.append(model_summary)
        else:
            if 'model' in config:
                model_summary = pd.DataFrame({'model': [model_id],
                                          'N features': '-',
                                          'N negative': '-',
                                          'learner': config['model'],
                                          'train_label': config['train_label_column']})
                summs.append(model_summary)
   
    if not len(summs) == 0:
        df_summ = pd.concat(summs)
        display(Markdown("Model summary"))
        display(HTML(df_summ[['model', 'N features', 'N negative', 'learner', 'train_label']].to_html(index=False, classes = ['sortable'],
                                                                                                      escape=False,
                                                                                                      float_format=int_or_float_format_func)))

summarize_models(model_list)

In [ ]:
def summarize_model_fit():
    fits = []
    for (model_id, config, csvdir) in model_list:
        model_fit_file = os.path.join(csvdir, '{}_model_fit.csv'.format(model_id))
        if os.path.exists(model_fit_file):
            fit = pd.read_csv(model_fit_file)
            fit['model'] = model_id
            fits.append(fit)
    if len(fits)>0:
        df_fit = pd.concat(fits)
        display(Markdown("## Model fit"))
        display(HTML(df_fit[['model', 'N responses','N features','R2','R2_adjusted']].to_html(index=False, classes = ['sortable'],
                                                                                              escape=False,
                                                                                             float_format=int_or_float_format_func)))
    
summarize_model_fit()